# Lab 6-1: Softmax Classifier

Author: Seungjae Lee (이승재)

여러개 중 하나를 예측할 때 유용하다

$$ S(y_i) = \frac{e^{y_i}}{\sum_j e^{y_j}} $$

## Softmax

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# For reproducibility
torch.manual_seed(1)

In [3]:
z = torch.FloatTensor([1, 2, 3])
out = F.softmax(z, dim=0)
print(out)

tensor([0.0900, 0.2447, 0.6652])


In [4]:
out.sum()

tensor(1.)

## Cross Entropy Loss

Softmax에 걸맞는 loss function

$$ L = \frac{1}{N} \sum y \log(\hat{y}) $$

In [5]:
pred = torch.randn(3, 5, requires_grad=True)
target = torch.randint(5, (3,), dtype=torch.int64)

In [6]:
print(pred)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519],
        [-0.1661, -1.5228,  0.3817, -1.0276, -0.5631],
        [-0.8923, -0.0583, -0.1955, -0.9656,  0.4224]], requires_grad=True)


In [7]:
print(target)

tensor([2, 3, 2])


In [8]:
F.cross_entropy(pred, target)

tensor(1.9087, grad_fn=<NllLossBackward>)

<div class="alert alert-warning">
    PyTorch에서 cross_entropy 는 softmax를 포함한다.
</div>

## Model

In [9]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3) # Output이 3!

    def forward(self, x):
        return self.linear(x)

In [10]:
x_train = [[1, 2, 1, 1]]
x_train = torch.FloatTensor(x_train)

In [11]:
model = SoftmaxClassifierModel()

In [12]:
prediction = model(x_train)
print(prediction)

tensor([[ 0.9147,  0.7679, -0.5841]], grad_fn=<ThAddmmBackward>)


In [13]:
F.softmax(prediction, dim=1)

tensor([[0.4792, 0.4138, 0.1070]], grad_fn=<SoftmaxBackward>)

## Training

In [14]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

이제 새로운 데이터로 코드를 돌려 피팅시켜보겠습니다.

In [15]:
# 모델 초기화
model = SoftmaxClassifierModel()
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs):
    
    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 100 == 99:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch+1, nb_epochs, cost.item()
        ))

Epoch  100/1000 Cost: 0.697610
Epoch  200/1000 Cost: 0.609269
Epoch  300/1000 Cost: 0.550272
Epoch  400/1000 Cost: 0.498888
Epoch  500/1000 Cost: 0.450523
Epoch  600/1000 Cost: 0.403491
Epoch  700/1000 Cost: 0.357054
Epoch  800/1000 Cost: 0.311161
Epoch  900/1000 Cost: 0.267986
Epoch 1000/1000 Cost: 0.241539


점점 $H(x)$ 의 $W$ 와 $b$ 를 조정해서 cost가 줄어드는 것을 볼 수 있습니다.

<div class="alert alert-warning">
    W 와 b 값을 출력해볼까?
</div>